In [1]:
import copy
from tictactoe import TicTacToe

class Node:
    def __init__(self, state, prev=None):
        self.prev = prev
        self.state = state
        self.nodes = []

In [2]:
root = Node(TicTacToe(), None)
current = root

for move_i, move_j in current.state.valid_moves():
    current.nodes.append(Node(copy.deepcopy(current.state), current))
    print(move_i, move_j)
    current.nodes[-1].state.make_move(move_i, move_j)
    

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


In [3]:
for node in current.nodes:
    print("Board:")
    node.state.print_board()
    print()

Board:
X| | 
-----
 | | 
-----
 | | 

Board:
 |X| 
-----
 | | 
-----
 | | 

Board:
 | |X
-----
 | | 
-----
 | | 

Board:
 | | 
-----
X| | 
-----
 | | 

Board:
 | | 
-----
 |X| 
-----
 | | 

Board:
 | | 
-----
 | |X
-----
 | | 

Board:
 | | 
-----
 | | 
-----
X| | 

Board:
 | | 
-----
 | | 
-----
 |X| 

Board:
 | | 
-----
 | | 
-----
 | |X

